In [ ]:
!git clone https://github.com/Holliemin9090/Mammographic-mass-CAD-via-pseudo-color-mammogram-and-Mask-R-CNN.git

fatal: destination path 'Mammographic-mass-CAD-via-pseudo-color-mammogram-and-Mask-R-CNN' already exists and is not an empty directory.


In [ ]:
import numpy as np
import cv2
from skimage.measure import label, regionprops


class Preprocess:
    def __init__(self, rawim, im, breast_mask, lesion_mask):
        self.raw = rawim
        self.image = im
        self.mask = breast_mask
        self.lesion_mask = lesion_mask
   
    def extract_breast_profile(image,lesion_mask, if_crop):
        
        breast_mask = np.zeros(np.shape(image))
        breast_mask[image>0]=1
        
        labelim = label(breast_mask)
        props =  regionprops(labelim)
#        find the largest object as the breast
        area = 0
        ind = 1
        for i in range(0,len(props)):
            if area<props[i].filled_area:
                area = props[i].filled_area
                ind = i+1
        breast_mask = np.zeros(np.shape(image))
        breast_mask[labelim==ind]=1  
        labelim = label(breast_mask)       
        props =  regionprops(labelim)
        boundingbox = props[0].bbox
#        crop the breast mask and mammogram
        if if_crop == 1:
            breast_mask = breast_mask[boundingbox[0]:boundingbox[2],boundingbox[1]:boundingbox[3]]
            breast_raw_image = image[boundingbox[0]:boundingbox[2],boundingbox[1]:boundingbox[3]]
            lesion_mask = lesion_mask[boundingbox[0]:boundingbox[2],boundingbox[1]:boundingbox[3]]
        else:
            breast_raw_image = image
#        breast_image = rescale2uint8(breast_raw_image,breast_mask)
        breast_image = rescale2uint16(breast_raw_image,breast_mask)
        return Preprocess(breast_raw_image,breast_image,breast_mask,lesion_mask)
    
def rescale2uint8(image,breast_mask):
    intensity_in_mask = image[breast_mask>0]
#    use top 0.2 percentile to do the strech
    maxi = np.percentile(intensity_in_mask,99.8)#np.max(intensity_in_mask)
    mini = np.percentile(intensity_in_mask,0.2)#np.min(intensity_in_mask)
#        stretch the image into 0~255
    
    image = 255*(image-mini)/(maxi-mini)
    image[breast_mask==0] = 0
    image[image<0] = 0
    image[image>255] = 255
    image = np.uint8(image)
          
    return image

def rescale2uint16(image,breast_mask):
    intensity_in_mask = image[breast_mask>0]
#    use top 0.2 percentile to do the strech
    maxi = np.percentile(intensity_in_mask,99.8)#np.max(intensity_in_mask)
    mini = np.percentile(intensity_in_mask,0.2)#np.min(intensity_in_mask)
#        stretch the image into 0~255
    
    image = 65535*(image-mini)/(maxi-mini)
    image[breast_mask==0] = 0
    image[image<0] = 0
    image[image>65535] = 65535
    image = np.uint16(image)
          
    return image


In [ ]:
import numpy as np
# To pad image into a square shape
def padimages(image,file_name, ratio):
    [length, width] = np.shape(image)
    if length/width>ratio:#1024/800
        print('This image needs padding.')
        add_wid = round(length*(1/ratio)-width)
        pad = np.zeros((length,add_wid))
        pad = pad.astype(image.dtype)
        if '_R_' in file_name:
        #                pad on the left
            pad_image = np.concatenate((pad,image),axis=1)
        else:
            pad_image = np.concatenate((image,pad),axis=1)
            
    return pad_image

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import data, color, io, img_as_float

def mask_overlay(img, mask):
    alpha = 0.5

    img = img_as_float(img)
    rows, cols = img.shape
    # Construct RGB version of grey-level image
    img_color = np.dstack((img, img, img))
    img_hsv = color.rgb2hsv(img_color)
    color_mask = np.zeros((rows, cols, 3))
    color_mask1 = color_mask[:,:,1]
    color_mask1[np.where(mask>0)] = 1
    color_mask[:,:,1] = color_mask1
    color_mask_hsv = color.rgb2hsv(color_mask)
    
    # Replace the hue and saturation of the original image
    # with that of the color mask
    img_hsv[..., 0] = color_mask_hsv[..., 0]
    img_hsv[..., 1] = color_mask_hsv[..., 1] * alpha
    
    img_masked = color.hsv2rgb(img_hsv)
    
    return img_masked

In [ ]:
%cd "/content/drive/MyDrive/Mammographic-mass-CAD-via-pseudo-color-mammogram-and-Mask-R-CNN-1"

/content/drive/MyDrive/Mammographic-mass-CAD-via-pseudo-color-mammogram-and-Mask-R-CNN-1


In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Mon Nov 26 15:46:32 2018

@author: Hang Min

Prepare the mammograms for pseudo-color transformation

To normalize the INbreast mammogram to 16bit

To pad the mammograms and masks to a square


"""



import os
import numpy as np
from Preprocess_mammo import Preprocess 
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from utility import padimages
#from skimage.segmentation import mark_boundaries
from skimage import io
from skimage.measure import label
from overlay import mask_overlay
import timeit

start = timeit.default_timer()

import warnings
warnings.filterwarnings("ignore")

#from skimage import data, color, io, img_as_float

image_path = "scans/raw_mammogram/"
annotation_path = 'scans/raw_annotation/'

save_image_path = "scans/preprocessed_image/"   
if not os.path.exists(save_image_path):        
    os.mkdir(save_image_path)
 
    
save_mask_path = "scans/preprocessed_mask/"    
if not os.path.exists(save_mask_path):        
    os.mkdir(save_mask_path)


file_names = os.listdir(image_path)    
file_names = sorted(file_names)


for i in range(0,len(file_names)):

    print(file_names[i])
    mammo = io.imread(image_path+file_names[i],0)
   
    lesion_mask = io.imread(annotation_path+file_names[i])
    
    if np.max(lesion_mask)>=0:
#    Extract the breast profile and crop the mammogram, breast mask and the lesion mask
#    Normalize the image into 16-bit
        breast_preprocess = Preprocess.extract_breast_profile(mammo,lesion_mask,1)
               
        mammo = breast_preprocess.image
        breast_mask = breast_preprocess.mask
        lesion_mask =breast_preprocess.lesion_mask    
        
        print ('Number of lesions: '+str(np.max(np.unique(label(lesion_mask)))))
        
#   pad the image, to ensure the aspect ratio is 1:1
        pad_mammo = padimages(mammo,file_names[i],1)
        
#    save the preprocessed image

        io.imsave(save_image_path + file_names[i],pad_mammo)
        
        #if the image has more than 1 lesion, then seperate them into different masks and number them.
        
        labelim = label(lesion_mask)
        if np.max(labelim)>0:
#            if there is at least 1 lesion.
            for l in range(1,np.max(labelim+1)):
                l_mask = np.zeros(np.shape(labelim))
                l_mask = l_mask.astype(lesion_mask.dtype)
                l_mask [labelim==l] = 255
                num_nonzero = np.where(l_mask>0)
                num_nonzero = len(num_nonzero[0])

                if num_nonzero>15:
                    print('A valid mask')
#                   Pad the mask in the same way as padding the image
                    pad_l_mask = padimages(l_mask,file_names[i],1)
                    io.imsave(save_mask_path+file_names[i][:-4]+str(l)+'.png',pad_l_mask)
                else:
                    print('Has a tiny piece of noise that is not valid for training!')
                    
        else:# if there is no lesion
            pad_lesion_mask = padimages(lesion_mask,file_names[i],1)
            io.imsave(save_mask_path+file_names[i][:-4]+str(0)+'.png',pad_lesion_mask)
       
stop = timeit.default_timer()
print('RunTime per image: ', (stop - start)/ len(file_names)) 

22580192_5530d5782fc89dd7_MG_R_CC_ANON.png
Number of lesions: 1
This image needs padding.
A valid mask
This image needs padding.
RunTime per image:  7.0066864209999835


In [ ]:
!git clone https://github.com/matterport/Mask_RCNN.git

fatal: destination path 'Mask_RCNN' already exists and is not an empty directory.


In [ ]:
import cv2
os.chdir('/content/drive/MyDrive/MaskRCNN-main/mrcnn/')
from Mask_RCNN.mrcnn import visualize
import Mask_RCNN.mrcnn.model as modellib

Using TensorFlow backend.


In [ ]:
print(ROOT_DIR)

/content/drive/MyDrive/Mammographic-mass-CAD-via-pseudo-color-mammogram-and-Mask-R-CNN-1


In [ ]:
%cd "//content//drive//MyDrive//Mammographic-mass-CAD-via-pseudo-color-mammogram-and-Mask-R-CNN-1"

/content/drive/MyDrive/Mammographic-mass-CAD-via-pseudo-color-mammogram-and-Mask-R-CNN-1


In [ ]:
ROOT_DIR = os.getcwd()

In [ ]:
!pip install keras==2.1.5

In [ ]:
!pip install tensorflow==1.14.0
import tensorflow as tf
print(tf.__version__)

In [ ]:

import os
import sys
import json
import datetime
import numpy as np
import skimage.draw
import matplotlib.image
import glob
import scipy.misc
from PIL import Image
#import imgaug 
from imgaug import augmenters as iaa

# Root directory of the project
ROOT_DIR = os.getcwd()
ROOT_DIR = ROOT_DIR+"/Mask_r_cnn/"

MAMOGRAM_IMAGE_DIR = "/scans/pseudo_color_image" #Path of the mammograms
MAMOGRAM_MASK_DIR = "/scans/preprocessed_mask/"# Path of the ground truth masks


# Import Mask RCNN
sys.path.append(ROOT_DIR) # To find local version of the library
from mrcnn.config import Config
from mrcnn import model as modellib, utils

# Path to trained weights file
COCO_WEIGHTS_PATH = os.path.join(ROOT_DIR, "mask_rcnn_balloon.h5")

# Directory to save logs and model checkpoints, if not provided
# through the command line argument --logs
DEFAULT_LOGS_DIR = os.path.join("/content/drive/MyDrive/Mammographic-mass-CAD-via-pseudo-color-mammogram-and-Mask-R-CNN-1/logs")#Log directory for saving the weights
DEMO_SAVE_DIR = "scans/seg_mask/"# path to save the segmentation masks
#if not os.path.exists(DEMO_SAVE_DIR):        
#    os.mkdir(DEMO_SAVE_DIR)

############################################################
#  Configurations
############################################################


class MamogramConfig(Config):
    """Configuration for training on the toy  dataset.
    Derives from the base Config class and overrides some values.
    """
    # Give the configuration a recognizable name
    NAME = "mamogram"

    # We use a GPU with 12GB memory, which can fit two images.
    # Adjust down if you use a smaller GPU.
    IMAGES_PER_GPU = 1

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # Background + lesion

    # Number of training steps per epoch,set to the number of training data here
    STEPS_PER_EPOCH = 100

    # Number of validation steps after each round of training
    VALIDATION_STEPS = 10
    # Resize mode: "none" or "square"

    IMAGE_RESIZE_MODE = "square"
    IMAGE_MIN_DIM = 1024
    IMAGE_MAX_DIM = 1024

    # Skip detections with < DETECTION_MIN_CONFIDENCE
    DETECTION_MIN_CONFIDENCE = 0.965 # alter this during testing to generate different TPR at different FPI
    # 0.7 0.75 0.8 0.85 0.9 

############################################################
#  Dataset
############################################################

class MamogramDataset(utils.Dataset):

    def load_mamogram(self, subset):
        """This method loads the actual image
        subset is either "train" or "val" depending on whether the image is part of the training or validation datasets 
        """
        # Add classes. We have only one class to add.
        # These are the things that will be segmented
        self.add_class("mamogram", 1, "lesion")

        # Train or validation dataset?
        #list all the files in the directory with the mamogram images
        files = os.listdir(ROOT_DIR + MAMOGRAM_IMAGE_DIR + subset )
        for fname in files:
            self.add_image("mamogram", image_id=fname, path=ROOT_DIR + MAMOGRAM_IMAGE_DIR + subset + fname, subset=subset, fname=fname)


    def load_mask(self, image_id):
        """load the instance masks for an image.
        Returns:
        a tuple containing:
        masks: A bool array of shape [height, width, instance count] with
        one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.

        use dtype=np.int32
        """
        info = self.image_info[image_id]
        fname = info['fname']
       

        files = glob.glob(ROOT_DIR + MAMOGRAM_MASK_DIR + info['subset'] + fname[0:-4] + "*")

        masks = []
        for i in range(0, len(files)):
            data = skimage.io.imread(files[i])
            
            if data.ndim != 1:
                data = skimage.color.rgb2gray(data)
          
            singleMask = data
            if i == 0:
                masks = np.zeros((singleMask.shape[0], singleMask.shape[1], len(files)))
            masks[:,:,i] = singleMask


        instanceMaskMap = np.array(np.ones([masks.shape[-1]], dtype=np.int32)) #this is VERY important: array of class ids in the order that they appear in bigdata
        # Return mask, and array of class IDs of each instance. Since we have
        # one class ID only, we return an array of 1s

        return (masks.astype(np.bool), instanceMaskMap)

    def load_image(self, image_id):
        """Load the specified image and return a [H,W,3] Numpy array.
		Taken from utils.py, any refinements we need can be done here
        """
        # Load image
        image = skimage.io.imread(self.image_info[image_id]['path'])
        # If grayscale. Convert to RGB for consistency.
        if image.ndim != 3:
            image = skimage.color.gray2rgb(image)
        # If has an alpha channel, remove it for consistency
        if image.shape[-1] == 4:
            image = image[..., :3]
        return image

    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        return info["path"]


def train(model):
    """Train the model."""
    # Training dataset.
    dataset_train = MamogramDataset()
    dataset_train.load_mamogram("/train/")
    dataset_train.prepare()

    # Validation dataset
    dataset_val = MamogramDataset()
    dataset_val.load_mamogram("/val/")
    dataset_val.prepare()

    # Image augmentation
    # http://imgaug.readthedocs.io/en/latest/source/augmenters.html
    aug = iaa.Sequential([
        iaa.OneOf([iaa.Fliplr(0.5),
                   iaa.Flipud(0.5),
                   iaa.Affine(rotate=90),
                   iaa.Affine(rotate=180),
                   iaa.Affine(rotate=270)]),
    ])

    # *** This training schedule is an example. Update to your needs ***
    # Since we're using a very small dataset, and starting from
    # COCO trained weights, we don't need to train too long. Also,
    # no need to train all layers, just the heads should do it.
    print("Training network heads")

    model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE,
                epochs=10,augmentation=aug,
                layers='all')
    
def segment(model, imPath):
    
    image = skimage.io.imread(imPath)

    fname = imPath.split('/')[-1]
    mrcnnData = model.detect([image], verbose=1)
       # documentation for model.detect:
       # """Runs the detection pipeline.

       # images: List of images, potentially of different sizes.

       # Returns a list of dicts, one dict per image. The dict contains:
       # rois: [N, (y1, x1, y2, x2)] detection bounding boxes
       # class_ids: [N] int class IDs
       # scores: [N] float probability scores for the class IDs
       # masks: [H, W, N] instance binary masks
       # """

    mrcnnData = mrcnnData[0] #model.detect takes a list of images, but here we only provide one image so the output is a list with just one element

    masks = mrcnnData['masks']
    for i in range(0, masks.shape[2]):
        #iterate through the masks
        maskSingle = np.squeeze(masks[:, :, i])
        file_name = DEMO_SAVE_DIR + "demo_mask_" + str(i) + "_" + fname + "_{:%Y%m%dT%H%M%S}.png".format(datetime.datetime.now())
        

        scipy.misc.imsave(file_name, maskSingle.astype(np.int64)) 


    print(mrcnnData)
    print("&&&&&&&&&&&: "+str(mrcnnData['rois']))
    print("&&&&&&&&&&&: "+str(mrcnnData['class_ids']))
    print("&&&&&&&&&&&: "+str(mrcnnData['scores']))

    return

def segmentWrapper(model, directory):
    """wrapper function for segment to take many images as an input, calls segment() on everything in the directory"""
    files = os.listdir(directory)
    for f in files:
        segment(model, directory + '/' + f)

def overlayResult(image, mask):
	"""Function to overlay segmentation mask on an image.
	usage: image_var = overlayResult(image, dict['masks'] || masks_var)
	
	image: RGB or grayscale image [height, width, 1 || 3].
	mask: segmentation mask [height, width, instance_count]
	
	returns resulting image.
	"""
	# Image is already in grayscale so we skip converting it
	# May need to create 3 dimensions if single dimension image though so
	# will add this as a placeholder
	gray = skimage.color.gray2rgb(skimage.color.rgb2gray(image)) * 255
	# Copy color pixels from the original color image where mask is set
	if mask.shape[-1] > 0:
		#collapse masks into one layer
		mask = (np.sum(mask, -1, keepdims=True) >= 1)
		overlay = np.where(mask, image, gray).astype(np.uint8)
	else:
		overlay = gray.astype(np.uint8)
		
	return overlay
	
config = MamogramConfig()

model = modellib.MaskRCNN(mode="training", config=config,
                                  model_dir=DEFAULT_LOGS_DIR)

weights_path = "/content/drive/MyDrive/Mammographic-mass-CAD-via-pseudo-color-mammogram-and-Mask-R-CNN-1/mask_rcnn_balloon.h5"

model.load_weights(weights_path, by_name=True)

TypeError: ignored

In [ ]:
%cd "/content/drive/MyDrive/Mammographic-mass-CAD-via-pseudo-color-mammogram-and-Mask-R-CNN-1"

/content/drive/MyDrive/Mammographic-mass-CAD-via-pseudo-color-mammogram-and-Mask-R-CNN-1


In [ ]:
train(model)

Training network heads

Starting at epoch 0. LR=0.001

Checkpoint Path: ROOT_DIR/logs/mamogram20210321T2014/mask_rcnn_mamogram_{epoch:04d}.h5
Selecting layers to train
conv1                  (Conv2D)
bn_conv1               (BatchNorm)
res2a_branch2a         (Conv2D)
bn2a_branch2a          (BatchNorm)
res2a_branch2b         (Conv2D)
bn2a_branch2b          (BatchNorm)
res2a_branch2c         (Conv2D)
res2a_branch1          (Conv2D)
bn2a_branch2c          (BatchNorm)
bn2a_branch1           (BatchNorm)
res2b_branch2a         (Conv2D)
bn2b_branch2a          (BatchNorm)
res2b_branch2b         (Conv2D)
bn2b_branch2b          (BatchNorm)
res2b_branch2c         (Conv2D)
bn2b_branch2c          (BatchNorm)
res2c_branch2a         (Conv2D)
bn2c_branch2a          (BatchNorm)
res2c_branch2b         (Conv2D)
bn2c_branch2b          (BatchNorm)
res2c_branch2c         (Conv2D)
bn2c_branch2c          (BatchNorm)
res3a_branch2a         (Conv2D)
bn3a_branch2a          (BatchNorm)
res3a_branch2b         (Conv

ERROR:root:Error processing image {'id': '22580192_5530d5782fc89dd7_MG_R_CC_ANON.png', 'source': 'mamogram', 'path': '/content/drive/MyDrive/Mammographic-mass-CAD-via-pseudo-color-mammogram-and-Mask-R-CNN-1/scans/pseudo_color_image/train/22580192_5530d5782fc89dd7_MG_R_CC_ANON.png', 'subset': '/train/', 'fname': '22580192_5530d5782fc89dd7_MG_R_CC_ANON.png'}
Traceback (most recent call last):
  File "/content/drive/MyDrive/MaskRCNN-main/mrcnn/model.py", line 1709, in data_generator
    use_mini_mask=config.USE_MINI_MASK)
  File "/content/drive/MyDrive/MaskRCNN-main/mrcnn/model.py", line 1212, in load_image_gt
    mask, class_ids = dataset.load_mask(image_id)
  File "<ipython-input-45-729ee543eb9f>", line 122, in load_mask
    instanceMaskMap = np.array(np.ones([masks.shape[-1]], dtype=np.int32)) #this is VERY important: array of class ids in the order that they appear in bigdata
AttributeError: 'list' object has no attribute 'shape'
ERROR:root:Error processing image {'id': '22580192_5530

Epoch 1/10


  File "<ipython-input-45-729ee543eb9f>", line 122, in load_mask
    instanceMaskMap = np.array(np.ones([masks.shape[-1]], dtype=np.int32)) #this is VERY important: array of class ids in the order that they appear in bigdata
AttributeError: 'list' object has no attribute 'shape'


AttributeError: ignored

In [ ]:
!pip install keras==2.1.5

In [ ]:
!pip install tensorflow==1.14.0
import tensorflow as tf
print(tf.__version__)

In [ ]:
"""
Original code: 
    
Mask R-CNN
Train on the toy Balloon dataset and implement color splash effect.

Copyright (c) 2018 Matterport, Inc.
Licensed under the MIT License (see LICENSE for details)
Written by Waleed Abdulla

------------------------------------------------------------
Adapted for mammographic mass detection and segmentation.


"""

import os
import sys
import json
import datetime
import numpy as np
import skimage.draw
import matplotlib.image
import glob
import scipy.misc
from PIL import Image
#import imgaug 
from imgaug import augmenters as iaa

# Root directory of the project
ROOT_DIR = os.getcwd()
ROOT_DIR = ROOT_DIR+"/Mask_r_cnn/"

MAMOGRAM_IMAGE_DIR = "/scans/pseudo_color_image/" #Path of the mammograms
MAMOGRAM_MASK_DIR = "/scans/preprocessed_mask/"# Path of the ground truth masks


# Import Mask RCNN
sys.path.append(ROOT_DIR) # To find local version of the library
from mrcnn.config import Config
from mrcnn import model as modellib, utils

# Path to trained weights file
COCO_WEIGHTS_PATH = os.path.join(ROOT_DIR, "mask_rcnn_balloon.h5")

# Directory to save logs and model checkpoints, if not provided
# through the command line argument --logs
DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")#Log directory for saving the weights
DEMO_SAVE_DIR = "scans/seg_mask/"# path to save the segmentation masks
if not os.path.exists(DEMO_SAVE_DIR):        
    os.mkdir(DEMO_SAVE_DIR)

############################################################
#  Configurations
############################################################


class MamogramConfig(Config):
    """Configuration for training on the toy  dataset.
    Derives from the base Config class and overrides some values.
    """
    # Give the configuration a recognizable name
    NAME = "mamogram"

    # We use a GPU with 12GB memory, which can fit two images.
    # Adjust down if you use a smaller GPU.
    IMAGES_PER_GPU = 1

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # Background + lesion

    # Number of training steps per epoch,set to the number of training data here
    STEPS_PER_EPOCH = 100

    # Number of validation steps after each round of training
    VALIDATION_STEPS = 10
    # Resize mode: "none" or "square"

    IMAGE_RESIZE_MODE = "square"
    IMAGE_MIN_DIM = 1024
    IMAGE_MAX_DIM = 1024

    # Skip detections with < DETECTION_MIN_CONFIDENCE
    DETECTION_MIN_CONFIDENCE = 0.965 # alter this during testing to generate different TPR at different FPI
    # 0.7 0.75 0.8 0.85 0.9 


############################################################
#  Dataset
############################################################

class MamogramDataset(utils.Dataset):

    def load_mamogram(self, subset):
        """This method loads the actual image
        subset is either "train" or "val" depending on whether the image is part of the training or validation datasets 
        """
        # Add classes. We have only one class to add.
        # These are the things that will be segmented
        self.add_class("mamogram", 1, "lesion")

        # Train or validation dataset?

        #list all the files in the directory with the mamogram images
        files = os.listdir(ROOT_DIR + MAMOGRAM_IMAGE_DIR + subset + "/")

        for fname in files:
            self.add_image("mamogram", image_id=fname, path=ROOT_DIR + MAMOGRAM_IMAGE_DIR + subset +"/"+ fname, subset=subset, fname=fname)


    def load_mask(self, image_id):
        """load the instance masks for an image.
        Returns:
        a tuple containing:
        masks: A bool array of shape [height, width, instance count] with
        one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.

        use dtype=np.int32
        """
        info = self.image_info[image_id]
        fname = info['fname']
       

        files = glob.glob(ROOT_DIR + MAMOGRAM_MASK_DIR + info['subset'] + fname[0:-4] + "*")

        masks = []
        for i in range(0, len(files)):
            data = skimage.io.imread(files[i])
            
            if data.ndim != 1:
                data = skimage.color.rgb2gray(data)
          
            singleMask = data
            if i == 0:
                masks = np.zeros((singleMask.shape[0], singleMask.shape[1], len(files)))
            masks[:,:,i] = singleMask

 

        instanceMaskMap = np.array(np.ones([masks.shape[-1]], dtype=np.int32)) #this is VERY important: array of class ids in the order that they appear in bigdata
        # Return mask, and array of class IDs of each instance. Since we have
        # one class ID only, we return an array of 1s

        return (masks.astype(np.bool), instanceMaskMap)

    def load_image(self, image_id):
        """Load the specified image and return a [H,W,3] Numpy array.
		Taken from utils.py, any refinements we need can be done here
        """
        # Load image
        image = skimage.io.imread(self.image_info[image_id]['path'])
        # If grayscale. Convert to RGB for consistency.
        if image.ndim != 3:
            image = skimage.color.gray2rgb(image)
        # If has an alpha channel, remove it for consistency
        if image.shape[-1] == 4:
            image = image[..., :3]
        return image

    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        return info["path"]


def train(model):
    """Train the model."""
    # Training dataset.
    dataset_train = MamogramDataset()
    dataset_train.load_mamogram("/train/")
    dataset_train.prepare()

    # Validation dataset
    dataset_val = MamogramDataset()
    dataset_val.load_mamogram("/val/")
    dataset_val.prepare()

    # Image augmentation
    # http://imgaug.readthedocs.io/en/latest/source/augmenters.html
    aug = iaa.Sequential([
        iaa.OneOf([iaa.Fliplr(0.5),
                   iaa.Flipud(0.5),
                   iaa.Affine(rotate=90),
                   iaa.Affine(rotate=180),
                   iaa.Affine(rotate=270)]),
    ])

    # *** This training schedule is an example. Update to your needs ***
    # Since we're using a very small dataset, and starting from
    # COCO trained weights, we don't need to train too long. Also,
    # no need to train all layers, just the heads should do it.
    print("Training network heads")

    model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE,
                epochs=10,augmentation=aug,
                layers='all')

def segment(model, imPath):
    
    image = skimage.io.imread(imPath)

    fname = imPath.split('/')[-1]
    mrcnnData = model.detect([image], verbose=1)
       # documentation for model.detect:
       # """Runs the detection pipeline.

       # images: List of images, potentially of different sizes.

       # Returns a list of dicts, one dict per image. The dict contains:
       # rois: [N, (y1, x1, y2, x2)] detection bounding boxes
       # class_ids: [N] int class IDs
       # scores: [N] float probability scores for the class IDs
       # masks: [H, W, N] instance binary masks
       # """

    mrcnnData = mrcnnData[0] #model.detect takes a list of images, but here we only provide one image so the output is a list with just one element

    masks = mrcnnData['masks']
    for i in range(0, masks.shape[2]):
        #iterate through the masks
        maskSingle = np.squeeze(masks[:, :, i])
        file_name = DEMO_SAVE_DIR + "demo_mask_" + str(i) + "_" + fname + "_{:%Y%m%dT%H%M%S}.png".format(datetime.datetime.now())
        

        scipy.misc.imsave(file_name, maskSingle.astype(np.int64)) 


    print(mrcnnData)
    print("&&&&&&&&&&&: "+str(mrcnnData['rois']))
    print("&&&&&&&&&&&: "+str(mrcnnData['class_ids']))
    print("&&&&&&&&&&&: "+str(mrcnnData['scores']))

    return

def segmentWrapper(model, directory):
    """wrapper function for segment to take many images as an input, calls segment() on everything in the directory"""
    files = os.listdir(directory)
    for f in files:
        segment(model, directory + '/' + f)

def overlayResult(image, mask):
	"""Function to overlay segmentation mask on an image.
	usage: image_var = overlayResult(image, dict['masks'] || masks_var)
	
	image: RGB or grayscale image [height, width, 1 || 3].
	mask: segmentation mask [height, width, instance_count]
	
	returns resulting image.
	"""
	# Image is already in grayscale so we skip converting it
	# May need to create 3 dimensions if single dimension image though so
	# will add this as a placeholder
	gray = skimage.color.gray2rgb(skimage.color.rgb2gray(image)) * 255
	# Copy color pixels from the original color image where mask is set
	if mask.shape[-1] > 0:
		#collapse masks into one layer
		mask = (np.sum(mask, -1, keepdims=True) >= 1)
		overlay = np.where(mask, image, gray).astype(np.uint8)
	else:
		overlay = gray.astype(np.uint8)
		
	return overlay
	
	
############################################################
#  Training
############################################################

if __name__ == '__main__':
    import argparse

    # Parse command line arguments
    parser = argparse.ArgumentParser(
        description='Train Mask R-CNN to detect breast lesions.')
    parser.add_argument("command",
                        metavar="<command>",
                        help="'train' or 'segment'")
    parser.add_argument('--weights', required=True,
                        metavar="/path/to/weights.h5",
                        help="Path to weights .h5 file or 'coco'")
    parser.add_argument('--image', required=False,
                        metavar='/path/to/image',
                        help="Path to image file for segmentation")
    args = parser.parse_args()

    # Configurations
    if args.command == "train":
        config = MamogramConfig()
    else:
        class InferenceConfig(MamogramConfig):
            # Set batch size to 1 since we'll be running inference on
            # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
            GPU_COUNT = 1
            IMAGES_PER_GPU = 1
        config = InferenceConfig()
    config.display()

    # Create model
    if args.command == "train":
        model = modellib.MaskRCNN(mode="training", config=config,
                                  model_dir=DEFAULT_LOGS_DIR)
    else:
        model = modellib.MaskRCNN(mode="inference", config=config,
                                  model_dir=DEFAULT_LOGS_DIR)

    # Select weights file to load
    if args.weights.lower() == "coco":
        weights_path = COCO_WEIGHTS_PATH
        # Download weights file
        if not os.path.exists(weights_path):
            utils.download_trained_weights(weights_path)
    else:
        weights_path = args.weights

    # Load weights
    print("Loading weights ", weights_path)
    if args.weights.lower() == "coco":
        # Exclude the last layers because they require a matching
        # number of classes  
        model.load_weights(weights_path, by_name=True, exclude=[
            "mrcnn_class_logits", "mrcnn_bbox_fc",
            "mrcnn_bbox", "mrcnn_mask"])
    else:
        model.load_weights(weights_path, by_name=True)

    # Train or evaluate
    if args.command == "train":
        train(model)
    elif args.command == "segment":
        if os.path.isdir(args.image):
            segmentWrapper(model, args.image)
        else:
            segment(model, args.image)
    else:
        print("'{}' is not recognized. "
              "Use 'train' or 'segment'".format(args.command))


In [ ]:
############ starting matlab codes  ###############

In [ ]:
#@mfunction("item_name")
def Read_files_in_folder(path, mode):
    # Read_files_in_folder Summary of this function goes here
    #   Detailed explanation goes here
    # This is a fuction that reads the files/folders under a folder path
    #   INPUT   path   the folder path
    #           mode   whether the user want read files or folders
    #   OUTPUT  item_name    return the names of the items under the path

    pt = (path)

    item_name = {}

    M = len(pt)

    k = 0
# format(mstring('short'))


    for i in range(1,M): #mslice[1:M]:
        if (pt(i) == ('.')):
            continue
        else:
            k = k + 1
            item_name{k}== pt
        end

    end

#end

SyntaxError: ignored

In [ ]:
image_path = 'scans\\preprocessed_image\\'
image_save_path = 'scans\\pseudo_color_image\\'


item_names = Read_files_in_folder(image_path, ('files'))

25
files


IndexError: ignored

In [ ]:
#@mfunction("len_bank")
def Morphological_filter_bank(Num_scale=None, D=None, type=None):
    #Morphological_filter_bank Summary of this function goes here
    #   This function generates the length of the linear structuring elements (LSE) 
    #   used in morphological filter elements on different scales. Either
    #   linear or logarithmic scale interval is used.
    #   INPUT     Num_scale  The number of scales used
    #             D          The diameter range of breast masses
    #             
    #             type       The scale type (linear or logarithmic)
    #   OUTPUT    len_bank    The magnitudes of the LSEs

    if strcmp(type, mstring('linear')):
        scale_interval = ceil((D(2) - D(1)) / Num_scale)
        len_bank = zeros(1, Num_scale + 1)
        for l in mslice[1:Num_scale + 1]:
            len_bank(l).lvalue = D(1) + (l - 1) * scale_interval
        end
        # This is a linear bank
        len_bank(Num_scale + 1).lvalue = D(2)

    end


    if strcmp(type, mstring('exponential')):
        scale_interval = (D(2) / D(1)) ** (1 / Num_scale)
        for l in mslice[1:Num_scale + 1]:
            len_bank(l).lvalue = round(D(1) * (scale_interval ** (l - 1)))
        end
        # This is a linear bank
        len_bank(Num_scale + 1).lvalue = D(2)

    end
#end

In [ ]:
#@mfunction("enhanced_image")
def Morphological_sifter(M1=None, M2=None, orientation=None, image=None, L_or_R=None, padding_option=None, breast_mask=None):#,rowmin,colmin
    # Summary of this function goes here
    #   Detailed explanation goes here
    # This is a function that does multi-scale morphological sifting used
    # linear structuring elements (LSE) with given length
    #INPUT  M1,M2: Length of the LSEs  M1>M2
    #       orientation: Orientations of the LSEs in degrees, it is a 1*N
    #       vector containing the angles of each LSE
    #       image: The input image to be processed
    #       L_or_R: Indicator of left or right breast
    #       padding_option: Image boundary padding options. 
    #                       If set to 0, pad the boundary with highest
    #                       intensity value.
    #                       If set to 1, pad it with replications of the pixels
    #                       on the boundary
    #       breast_mask: The binary breast mask
    #OUTPUT enhanced_image: The output image from MMS
    #       


    newimage = image
    [m, n] = size(newimage)

    #% Border effect control: border padding
    # Option 1: pad with highest pixel value
    temp = uint16(65535 * ones(m + 4 * M1, n + 4 * M1))
    temp(mslice[2 * M1 + 1:2 * M1 + m], mslice[2 * M1 + 1:2 * M1 + n]).lvalue = newimage# Add white margins to each side of the image to prevent edge effect of morphological process

    # Option 2: replicate the pixels on the border
    if padding_option == 1:
        if L_or_R == 1:        # left breast
            edge = newimage(mslice[:], mslice[1:min(n, 2 * M1)])
            temp(mslice[2 * M1 + 1:2 * M1 + m], mslice[2 * M1 - size(edge, 2) + 1:2 * M1]).lvalue = fliplr(edge)
        else:        # right breast
            edge = newimage(mslice[:], mslice[max(1, n - 2 * M1 + 1):n])
            temp(mslice[2 * M1 + 1:2 * M1 + m], mslice[n + 2 * M1 + 1:n + 2 * M1 + size(edge, 2)]).lvalue = fliplr(edge)
        end
    end
    #% Apply multi-scale morphological sifting

    enhanced_image = zeros(size(temp))
    for k in mslice[1:length(orientation)]:
        B1 = strel(mstring('line'), M1, orientation(k))
        B2 = strel(mstring('line'), M2, orientation(k))
        bg1 = imopen(temp, B1)
        r1 = imsubtract(temp, bg1)
        r2 = imopen(r1, B2)
        enhanced_image = enhanced_image + double(r2)

    end

    enhanced_image = enhanced_image(mslice[2 * M1 + 1:2 * M1 + m], mslice[2 * M1 + 1:2 * M1 + n])# Reset the image into the original size
    [enhanced_image] = Normalization_mask(enhanced_image, breast_mask, 8)

#end

In [ ]:
#@mfunction("new_im")
def Normalization_mask(image=None, mask=None, mode=None):
    #UNTITLED5 Summary of this function goes here
    #   Detailed explanation goes here
    image = double(image)
    inten = image(mask == 1)
    mini = min(inten)
    maxi = max(inten)

    image = (image - mini) /eldiv/ abs(maxi - mini)
    image(mask < 1).lvalue = 0
    if mode == 8:
        new_im = uint8(image * 255)

    elif mode == 16:
        new_im = uint16(image * 65535)

    elif strcmp(mode, string('double')):
        new_im = double(image)
    end
#end

In [ ]:
# Try to feed in gradient and morphological filtered images in to different
# channal


image_path = 'scans\\preprocessed_image\\'
image_save_path = 'scans\\pseudo_color_image\\'


item_names = Read_files_in_folder(image_path, ('files'))

mass_size_range_mm = mcat([15, 3689])# square mm
resolution = 0.07# spatial resolution of the INbreast mammograms, 0.07mm
resize_ratio = 1 / 4
mass_diameter_range_pixel = mcat([floor((mass_size_range_mm(1) / pi) ** 0.5 * 2 / (resolution / resize_ratio)), ceil((mass_size_range_mm(2) / pi) ** 0.5 * 2 / (resolution / resize_ratio))])# diameter range in pixels

for i in mslice[1:length(item_names)]:
    close(mstring('all'))
    disp(item_names(i))
    image = imread(strcat(image_path, item_names(i)))

    #% Image subsampling using 2 level db2 wavelet
    image = image(mslice[:], mslice[:], 1)
    breast_mask = (image > 0)
    dwt2(image, mstring('db2'))
    dwt2(cA, mstring('db2'))

    dwt2(breast_mask, mstring('db2'))
    dwt2(cA, mstring('db2'))
    breast_mask = (breast_mask >= 1)


    # Normalize the grayscale image
    [new_im] = Normalization_mask(image, breast_mask, 8)
    #     figure,imshow(new_im);

    #% Apply multi-scale morphological sifting and append the images from 2 scales to the grayscale mammogram
    L_OR_R = isempty(strfind(item_names(i), mstring('_R_')))# check if it is a left or right breast
    CC_OR_ML = isempty(strfind(item_names(i), mstring('_CC_')))
    degree_bank = mslice[0:10:170]# The orientations of the linear structuring elements (LSEs)
    Num_scale = 2# Using 2 scales
    # Generate the length for LSEs on different scales
    [len_bank] = Morphological_filter_bank(Num_scale, mass_diameter_range_pixel, mstring('exponential'))
    enhanced_image = mcellarray([])
    for j in mslice[1:Num_scale]:
        # Boundary padding
        padding_mode = 1    #
        if j == 1 or CC_OR_ML == 1:
            #           if it is a small scale or it is a MLO view
            padding_mode = 0        # highest value padding
        end
        enhanced_image[j] = Morphological_sifter(len_bank(j + 1), len_bank(j), degree_bank, new_im, L_OR_R, padding_mode, breast_mask)
        #
    end
    Pseudo_color_im = cat(3, new_im, enhanced_image(1), enhanced_image(2))
    figure
    imshow(Pseudo_color_im)

    imwrite(Pseudo_color_im, strcat(image_save_path, item_names(i)))
end
elapsedTime = toc

#% Process the annotation masks, so that they are the same size as the mammograms
anno_path = mstring('scans\\preprocessed_mask\\')
anno_save_path = mstring('scans\\preprocessed_mask1\\')
if not exist(anno_save_path, mstring('dir')):
    mkdir(anno_save_path)
end

item_names = Read_files_in_folder(anno_path, mstring('files'))
for i in mslice[1:length(item_names)]:
    anno = imread(strcat(anno_path, item_names(i)))
    anno(anno == 255).lvalue = 1
    anno = double(anno)
    dwt2(anno, mstring('db2'))
    dwt2(cA, mstring('db2'))
    anno = abs(anno)
    anno(anno >= 1).lvalue = 255
    anno(anno < 1).lvalue = 0
    anno = uint8(anno)
    imwrite(anno, strcat(anno_save_path, item_names(i)))
end


NameError: ignored

In [ ]:
!apt install octave

In [ ]:
!octave -W '/content/Mammographic-mass-CAD-via-pseudo-color-mammogram-and-Mask-R-CNN/Read_files_in_folder.m'

In [ ]:
!octave -W '/content/Mammographic-mass-CAD-via-pseudo-color-mammogram-and-Mask-R-CNN/Morphological_filter_bank.m'

error: Invalid call to type.  Correct usage is:

 -- type NAME ...
 -- type -q NAME ...
 -- text = type ("NAME", ...)
error: called from
    print_usage at line 91 column 5
    type at line 38 column 5
    Morphological_filter_bank at line 12 column 1


In [ ]:
!octave -W '/content/Mammographic-mass-CAD-via-pseudo-color-mammogram-and-Mask-R-CNN/Morphological_sifter.m'

In [ ]:
!octave -W '/content/Mammographic-mass-CAD-via-pseudo-color-mammogram-and-Mask-R-CNN/Normalization_mask.m'

In [ ]:
!octave -W '/content/Mammographic-mass-CAD-via-pseudo-color-mammogram-and-Mask-R-CNN/Pseudo_color_image_generation.m'

In [ ]:
!octave -W '/content/Mammographic-mass-CAD-via-pseudo-color-mammogram-and-Mask-R-CNN/Pseudo_color_image_generation.m'

In [ ]:
!octave -W '/content/Mammographic-mass-CAD-via-pseudo-color-mammogram-and-Mask-R-CNN/Pseudo_color_image_generation.m'


{
  [1,1] = 22580192_5530d5782fc89dd7_MG_R_CC_ANON.png
}
ami
22580192_5530d5782fc89dd7_MG_R_CC_ANON.png
error: 'dwt2' undefined near line 29 column 17
error: called from
    Pseudo_color_image_generation at line 29 column 15


In [ ]:
pip install PyWavelets==0.2.2

  Using cached https://files.pythonhosted.org/packages/34/23/55501cba73984d1909a67170677b6a92b152448fca680ff062e40acd28f4/PyWavelets-0.2.2.zip
ERROR: Command errored out with exit status 1: python setup.py egg_info Check the logs for full command output.


In [ ]:
!python untitled.py

  File "untitled.py", line 1
    pip install PyWavelets
              ^
SyntaxError: invalid syntax


In [ ]:
pip install PyWavelets

In [ ]:
% Try to feed in gradient and morphological filtered images in to different
% channal
clc,clear,close all;
tic

image_path = 'scans\preprocessed_image\';
image_save_path = 'scans\pseudo_color_image\';

if ~exist(image_save_path,'dir')
    mkdir(image_save_path)
end
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
item_names = Read_files_in_folder( image_path, 'files' ); 

mass_size_range_mm = [15 3689];% square mm
resolution = 0.07;% spatial resolution of the INbreast mammograms, 0.07mm
resize_ratio = 1/4;
mass_diameter_range_pixel = [floor((mass_size_range_mm(1)/pi)^0.5*2/(resolution/resize_ratio)),...
    ceil((mass_size_range_mm(2)/pi)^0.5*2/(resolution/resize_ratio))];% diameter range in pixels

for i = 1:length(item_names)
    close all;
    disp(item_names{i});
    image = imread(strcat(image_path,item_names{i}));
    
    %% Image subsampling using 2 level db2 wavelet
    image = image(:,:,1);
    breast_mask = (image>0);
    [cA,~,~,~] = dwt2(image,'db2');
    [image,~,~,~] = dwt2(cA,'db2');
    
    [cA,~,~,~] = dwt2(breast_mask,'db2');
    [breast_mask,~,~,~] = dwt2(cA,'db2');
    breast_mask = (breast_mask>=1);
    
    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    % Normalize the grayscale image
    [new_im] = Normalization_mask(image,breast_mask,8);
    %     figure,imshow(new_im);
    
    %% Apply multi-scale morphological sifting and append the images from 2 scales to the grayscale mammogram
    L_OR_R = isempty(strfind(item_names{i},'_R_'));% check if it is a left or right breast
    CC_OR_ML = isempty(strfind(item_names{i},'_CC_'));
    degree_bank = 0:10:170;% The orientations of the linear structuring elements (LSEs)
    Num_scale = 2; % Using 2 scales
    % Generate the length for LSEs on different scales
    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%5
    [ len_bank ] = Morphological_filter_bank( Num_scale, mass_diameter_range_pixel, 'exponential' );
    enhanced_image = {};
    for j = 1:Num_scale
        % Boundary padding
        padding_mode = 1;%
        if j==1||CC_OR_ML==1
            %           if it is a small scale or it is a MLO view
            padding_mode = 0;% highest value padding
        end
        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
        [enhanced_image{j}] = Morphological_sifter(len_bank(j+1),len_bank(j),degree_bank,new_im,L_OR_R, padding_mode, breast_mask);
        %
    end
    Pseudo_color_im = cat(3,new_im,enhanced_image{1},enhanced_image{2});
        figure,imshow(Pseudo_color_im);
        imwrite(Pseudo_color_im,strcat(image_save_path,item_names{i}));
end
elapsedTime = toc;

%% Process the annotation masks, so that they are the same size as the mammograms
anno_path = 'scans\preprocessed_mask\';
anno_save_path = 'scans\preprocessed_mask1\';
if ~exist(anno_save_path,'dir')
    mkdir(anno_save_path)
end

item_names = Read_files_in_folder( anno_path, 'files' );
for i = 1:length(item_names)
    anno = imread(strcat(anno_path,item_names{i}));
    anno(anno==255) = 1;
    anno= double(anno);
    [cA,~,~,~] = dwt2(anno,'db2');
    [anno,~,~,~] = dwt2(cA,'db2');
    anno = abs(anno);
    anno(anno>=1) = 255;
    anno(anno<1) = 0;
    anno = uint8(anno);
        imwrite(anno,strcat(anno_save_path,item_names{i}));
end

SyntaxError: ignored

In [ ]:

pt=dir(image_path);

item_names = {}; 

M=length(pt);

k = 0;

format short


for i = 1 : M
    if strcmp(pt (i).name, '.') || strcmp(pt (i).name, '..')||(pt(i).isdir==0 && strcmp('files','folder'))
        continue;
    else
            k = k + 1;
            item_names{k} = pt (i).name;
    end
    
end

In [ ]:
image_path= '/content/Mammographic-mass-CAD-via-pseudo-color-mammogram-and-Mask-R-CNN/scans/Pseudo_color_image_generation.m'

In [ ]:
os.path.exists(image_path)

True

In [ ]:
% Root directory of the project
ROOT_DIR = os.getcwd()

ROOT_DIR = ROOT_DIR+'/content/Mammographic-mass-CAD-via-pseudo-color-mammogram-and-Mask-R-CNN/scans/preprocessed_image/'

In [ ]:
print(ROOT_DIR)

/content/content/Mammographic-mass-CAD-via-pseudo-color-mammogram-and-Mask-R-CNN/scans/preprocessed_image/


In [ ]:
% Try to feed in gradient and morphological filtered images in to different
% channal
clc,clear,close all;
tic


image_path = '/content/Mammographic-mass-CAD-via-pseudo-color-mammogram-and-Mask-R-CNN/scans/preprocessed_image/';

image_save_path = '/content/Mammographic-mass-CAD-via-pseudo-color-mammogram-and-Mask-R-CNN/scans/pseudo_color_image/';

if ~exist(image_save_path,'dir')
    mkdir(image_save_path)
end

pt=dir(image_path);

item_names = {}; 

M=length(pt);

k = 0;

format short


for i = 1 : M
    if strcmp(pt (i).name, '.') || strcmp(pt (i).name, '..')||(pt(i).isdir==0 && strcmp('files','folder'))
        continue;
    else
            k = k + 1;
            item_names{k} = pt (i).name;
    end
    
end
disp(item_names)
%item_names = Read_files_in_folder( image_path, 'files' );

%caPathList = genpath('/content/Mammographic-mass-CAD-via-pseudo-color-mammogram-and-Mask-R-CNN/scans')
%disp(caPathList)
%Files=dir(image_path);
%for k=1:length(Files)
   %item_names=strcmp(Files(k).name, '..')
   %item_names= Files(k).isdir==0 && strcmp('files','folder')
%end
%(pt(i).isdir==0 && strcmp(mode,'folder'))




mass_size_range_mm = [15 3689];% square mm
resolution = 0.07;% spatial resolution of the INbreast mammograms, 0.07mm
resize_ratio = 1/4;
mass_diameter_range_pixel = [floor((mass_size_range_mm(1)/pi)^0.5*2/(resolution/resize_ratio)),...
    ceil((mass_size_range_mm(2)/pi)^0.5*2/(resolution/resize_ratio))];% diameter range in pixels

for i = 1:length(item_names)
    close all;
    disp(item_names{i});
    image = imread(strcat(image_path,item_names{i}));
    %disp(length(image))
    
    %% Image subsampling using 2 level db2 wavelet
    image = image(:,:,1);
    breast_mask = (image>0);
    [cA,~,~,~] = dwt2(image,'db2');
    [image,~,~,~] = dwt2(cA,'db2');
    
    [cA,~,~,~] = dwt2(breast_mask,'db2');
    [breast_mask,~,~,~] = dwt2(cA,'db2');
    breast_mask = (breast_mask>=1);
    
    
    % Normalize the grayscale image
    [new_im] = Normalization_mask(image,breast_mask,8);
    %     figure,imshow(new_im);
    
    %% Apply multi-scale morphological sifting and append the images from 2 scales to the grayscale mammogram
    L_OR_R = isempty(strfind(item_names{i},'_R_'));% check if it is a left or right breast
    CC_OR_ML = isempty(strfind(item_names{i},'_CC_'));
    degree_bank = 0:10:170;% The orientations of the linear structuring elements (LSEs)
    Num_scale = 2; % Using 2 scales
    % Generate the length for LSEs on different scales
    [ len_bank ] = Morphological_filter_bank( Num_scale, mass_diameter_range_pixel, 'exponential' );
    enhanced_image = {};
    for j = 1:Num_scale
        % Boundary padding
        padding_mode = 1;%
        if j==1||CC_OR_ML==1
            %           if it is a small scale or it is a MLO view
            padding_mode = 0;% highest value padding
        end
        [enhanced_image{j}] = Morphological_sifter(len_bank(j+1),len_bank(j),degree_bank,new_im,L_OR_R, padding_mode, breast_mask);
        %
    end
    Pseudo_color_im = cat(3,new_im,enhanced_image{1},enhanced_image{2});
        figure,imshow(Pseudo_color_im);
        imwrite(Pseudo_color_im,strcat(image_save_path,item_names{i}));
end
elapsedTime = toc;

%% Process the annotation masks, so that they are the same size as the mammograms
anno_path = 'scans\preprocessed_mask\';
anno_save_path = 'scans\preprocessed_mask1\';
if ~exist(anno_save_path,'dir')
    mkdir(anno_save_path)
end

item_names = Read_files_in_folder( anno_path, 'files' );
for i = 1:length(item_names)
    anno = imread(strcat(anno_path,item_names{i}));
    anno(anno==255) = 1;
    anno= double(anno);
    [cA,~,~,~] = dwt2(anno,'db2');
    [anno,~,~,~] = dwt2(cA,'db2');
    anno = abs(anno);
    anno(anno>=1) = 255;
    anno(anno<1) = 0;
    anno = uint8(anno);
        imwrite(anno,strcat(anno_save_path,item_names{i}));
end

In [ ]:
function [ item_names ] = Read_files_in_folder( path, mode )
% Read_files_in_folder Summary of this function goes here
%   Detailed explanation goes here
% This is a fuction that reads the files/folders under a folder path
%   INPUT   path   the folder path
%           mode   whether the user want read files or folders
%   OUTPUT  item_name    return the names of the items under the path

pt=dir(path);

item_names = {}; 

M=length(pt);

k = 0;

format short


for i = 1 : M
    if strcmp(pt (i).name, '.') || strcmp(pt (i).name, '..')||(pt(i).isdir==0 && strcmp('files','folder'))
        continue;
    else
            k = k + 1;
            item_names{k} = pt (i).name;
    end
    
end


    

end



SyntaxError: ignored